In [59]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# ==================== 1. ЗАГРУЗКА ДАННЫХ ====================
url = 'C:/Users/6muni/Documents/adult.csv'  # ссылка на датасет (Adult Income)
adult = pd.read_csv(url)
# print(adult.info())

# ==================== 2. ПРЕОБРАЗОВАНИЕ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ ====================
# Преобразуем в бинарный формат
adult['income'] = adult['income'].str.strip().map({'<=50K': 0, '>50K': 1})

print(f"\nУникальные значения после преобразования: {adult['income'].unique()}")
print(f"Пропущенные значения: {adult['income'].isna().sum()}")

# Проверяем баланс классов
class_distribution = adult['income'].value_counts(normalize=True) * 100
print(f"\nРаспределение классов в исходных данных:")
print(f"  Класс 0 (<=50K): {class_distribution[0]:.2f}% ({adult['income'].value_counts()[0]} samples)")
print(f"  Класс 1 (>50K):  {class_distribution[1]:.2f}% ({adult['income'].value_counts()[1]} samples)")

# ==================== 3. РАЗДЕЛЕНИЕ НА ПРИЗНАКИ И ЦЕЛЕВУЮ ====================
# Разделяем на признаки (X) и целевую переменную (y)
X = adult.drop('income', axis=1)
y = adult['income']

# ==================== 4. РАЗДЕЛЕНИЕ НА TRAIN/TEST ====================
# Разделение на train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,  # 20% на тест
    random_state=42,  # для воспроизводимости
    stratify=y  # стратификация по income
)

# ==================== 5. ПРОВЕРКА РАЗМЕРОВ ====================
print(
    f"\nРазмеры выборок для X_train: {X_train.shape} ({(X_train.shape[0] / len(adult) * 100):.1f}% от исходных), y_train: {y_train.shape}")
print(
    f"Размеры выборок для X_train: {X_test.shape} ({(X_test.shape[0] / len(adult) * 100):.1f}% от исходных), y_train: {y_test.shape}")

# ==================== 6. ПРОВЕРКА STRATIFY ====================
print("\nПроверка stratify:")
print(f"Оригинал: {y.value_counts(normalize=True).values}")
print(f"Train:    {y_train.value_counts(normalize=True).values}")
print(f"Test:     {y_test.value_counts(normalize=True).values}")


# TODO:
# 1) Правильно разделить на train/test (80/20)
# 2) Проверить stratify (для classification)
# 3) Установить random_state для воспроизводимости
# 4) Проверить размеры и распределение классов


Уникальные значения после преобразования: [0 1]
Пропущенные значения: 0

Распределение классов в исходных данных:
  Класс 0 (<=50K): 76.07% (37155 samples)
  Класс 1 (>50K):  23.93% (11687 samples)

Размеры выборок для X_train: (39073, 14) (80.0% от исходных), y_train: (39073,)
Размеры выборок для X_train: (9769, 14) (20.0% от исходных), y_train: (9769,)

Проверка stratify:
Оригинал: [0.76071823 0.23928177]
Train:    [0.76072992 0.23927008]
Test:     [0.76067151 0.23932849]


In [89]:
# Копируем данные для преобразований
X_processed = X.copy()

# Предварительная обработка столбцов
cat_cols_original = X.select_dtypes(include=['object']).columns.tolist()
print(f"\nКатегориальные признаки ДО: \n\t{cat_cols_original}")
num_cols_original = X.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nЧисловые признаки ДО: \n\t{num_cols_original}")

# ==================== 1. POLYNOMIAL FEATURES ====================
X_processed['age_squared'] = X_processed['age'] ** 2
X_processed['hours_squared'] = X_processed['hours-per-week'] ** 2

# ==================== 2. INTERACTION FEATURES ====================
X_processed['age_hours'] = X_processed['age'] * X_processed['hours-per-week']

# ==================== 3. BINNING ====================
age_bins = [0, 25, 45, 65, 100]
age_labels = ['young', 'middle', 'senior', 'elder']
X_processed['age_bin'] = pd.cut(X_processed['age'], bins=age_bins, labels=age_labels, include_lowest=True)

# ==================== 4. ONE-HOT ENCODING ====================
# Применяем one-hot к ВСЕМ категориальным признакам (включая созданный age_bin)
X_processed = pd.get_dummies(X_processed, drop_first=True)

# ==================== 5. LABEL ENCODING (пример, если нужно) ====================
# Уместно, если есть естественный порядок внутри категории. Пример:
# education = ["Preschool", "HighSchool", "Bachelors"] → можно закодировать как 0,1,2.
# Неуместно, когда порядок искусственный.

# Создадим отдельный DataFrame с label encoding ТОЛЬКО для education
# (перед one-hot кодированием нужно было бы сохранить оригинальный education)
# Примечание: Label Encoding делаем ДО one-hot, если нужен

# Пример для education:
# education_map = {'Preschool':0, 'HS-grad':1, 'Bachelors':2, ...}
# X_processed['education_label'] = X['education'].map(education_map)


# Итог
num_cols_processed = X_processed.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nИтого: {X_processed.shape[1]} признаков после преобразований")
print(
    f"\nКатегориальные признаки ПОСЛЕ: \n\t{len(cat_cols_original)} категориальных признаков преобразованы в {X_processed.shape[1] - len(num_cols_original) - 3} бинарных признака")
print(f"\nЧисловые признаки ПОСЛЕ: \n\t{num_cols_processed}")


# TODO: Создать новые признаки
# 1) Polynomial features
# 2) Interaction features (A * B, A / B)
# 3) Binning (категоризация непрерывных признаков)
# 4) One-hot encoding для категориальных
# 5) Label encoding (когда это ok, когда нет)


Категориальные признаки ДО: 
	['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']

Числовые признаки ДО: 
	['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']

Итого: 106 признаков после преобразований

Категориальные признаки ПОСЛЕ: 
	8 категориальных признаков преобразованы в 97 бинарных признака

Числовые признаки ПОСЛЕ: 
	['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'age_squared', 'hours_squared', 'age_hours']


In [93]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Определяем числовые и категориальные признаки (из оригинальных данных)
num_cols_original = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_original = X.select_dtypes(include=['object']).columns.tolist()

# Преобразования
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(drop='first', handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols_original),
        ('cat', cat_transformer, cat_cols_original)
    ]
)

# Полный pipeline с моделью
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# Обучаем одной командой
pipeline.fit(X_train, y_train)

# Итог
print("Pipeline успешно создан и обучен на train-данных.")
print(f"Score на train: {pipeline.score(X_train, y_train):.4f}")
print(f"Score на test:  {pipeline.score(X_test, y_test):.4f}")

# TODO: Создать pipeline
# 1) Для числовых: StandardScaler
# 2) Для категориальных: OneHotEncoder
# 3) Объединить в ColumnTransformer
# 4) Добавить модель в конец pipeline
# 5) Обучить одной командой: pipeline.fit(X_train, y_train)

Pipeline успешно создан и обучен на train-данных.
Score на train: 0.8537
Score на test:  0.8538
